Install the necessary libraries and SDKs.

In [ ]:
!pip install semantic-kernel
!pip install psycopg
!pip install psycopg_pool
!pip install psycopg[binary,pool]

Import the necessary classes.

In [ ]:
from typing import Tuple
import psycopg2
import psycopg

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    OpenAITextEmbedding,
    AzureChatCompletion,
    AzureTextEmbedding,
)

Create a new Semantic Kernel object.

In [ ]:
kernel = sk.Kernel()

Setup the completions and embeddings service.

In [ ]:
completions_deployment_name = 'completions4'
embeddings_deployment_name = 'embeddings'

azure_endpoint = "YOUR_ENDPOINT"
azure_key = 'YOUR_KEY'

kernel.add_service(
    AzureChatCompletion(deployment_name=completions_deployment_name, endpoint=azure_endpoint, api_key=azure_key),
)

Create a new memory store using PostgreSQL, be sure to replace the `SUFFIX`.

In [ ]:
from semantic_kernel.connectors.memory.postgres import PostgresMemoryStore

#connect to the database
username = 'wsuser'
password = 'Solliance123'
host = 'pgsqldevSUFFIXflex14.postgres.database.azure.com'
port = '5432'
dbname = 'ailabs'

connection_string = f"postgresql://{username}:{password}@{host}:{port}/{dbname}"

memory_store = PostgresMemoryStore(connection_string=connection_string, default_dimensionality=1536, min_pool=1, max_pool=10)

memory_store.create_collection(collection_name='aboutMe')

Add a text embedding service and register the memory store with Semantic Kernel.

In [ ]:
embeddings = AzureTextEmbedding(deployment_name=embeddings_deployment_name, endpoint=azure_endpoint, api_key=azure_key)

kernel.add_service(embeddings)

In [ ]:
from semantic_kernel.core_plugins import TextMemoryPlugin
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory

memory = SemanticTextMemory(storage=memory_store, embeddings_generator=embeddings)

kernel.import_plugin_from_object(TextMemoryPlugin(memory), "TextMemoryPlugin")

Define the populate_memory function.

In [ ]:
async def populate_memory(memory) -> None:
    # Add some documents to the semantic memory
    await memory.save_information(collection="aboutMe", id="info1", text="My name is Andrea")
    await memory.save_information(
        collection="aboutMe", id="info2", text="I currently work as a tour guide"
    )
    await memory.save_information(
        collection="aboutMe", id="info3", text="I've been living in Seattle since 2005"
    )
    await memory.save_information(
        collection="aboutMe",
        id="info4",
        text="I visited France and Italy five times since 2015",
    )
    await memory.save_information(collection="aboutMe", id="info5", text="My family is from New York")

Call the populate_memory function to add data to PostgreSQL.

In [ ]:
await populate_memory(memory)

# Review the memory data

- Run the following query against the Azure Database for PostgreSQL Flexible Server to see the items added to the database:

In [ ]:
select *
from "aboutMe"

Define the search_memory_examples function.

In [ ]:
async def search_memory_examples(memory) -> None:
    questions = [
        "what's my name",
        "where do I live?",
        "where's my family from?",
        "where have I traveled?",
        "what do I do for work",
    ]

    for question in questions:
        print(f"Question: {question}")
        result = await memory.search("aboutMe", question)

        try:
            print(f"Answer: {result[0].text}\n")
        except IndexError:
            print("No answer found\n")

Search the memory for data.

In [ ]:
await search_memory_examples(memory)